In [1]:
import numpy as np
import cmath

In [16]:
J = 3 # the number of harmonics to take in the computation of side bands 
wp = 11.561
ws = 10
Ap0 = 1
I_star = 10
L = 1.

def dispersion(w):
    return w # assume no dispersion

In [17]:
# create wn from n
def wn_cal(n, wp=wp, ws=ws):
    return n*wp + ws - wp

def del_kn_cal(n, wp=wp, ws=ws):
    kn = dispersion(wn_cal(n, wp=wp, ws=ws))
    kn_minus_2 = dispersion(wn_cal(n-2, wp=wp, ws=ws))
    kp = dispersion(wp)
    return kn - kn_minus_2 - 2*kp
    

# create a dictionary of vn's 
def vn_gen(n, J=J):
    vn = np.empty(J+1)
    for num in range(J+1):
        j = num+1 # the j in Eq. 9a
        if 1 <= j <= 1 + (n+J)/2:
            vn[num]= 1
        if 1 + (n+J)/2 < j <= J+1:
            vn[num]=-1
    return vn

In [18]:
# calculate delta phi
del_phi = dispersion(wp)*Ap0**2/(8*I_star**2)

# calculate K
K = np.empty(J+1)
K[0] = 0
for num in range(1, J+1):
    K[num] = del_kn_cal(-J + 2*num) + 2*del_phi

In [19]:
# construct Coefficient matrix for the linear ODEs for Bn
# this is a (2J+1)*(2J+1) matrix, corresponding to -J,...,0,...,+J

CoefficientMatrix = np.zeros([2*J+1,2*J+1], dtype=np.complex_)

for num in range(2*J+1):
    n = num - J
    if n%2 == 0:
        continue
    if n-2 >= -J: 
        CoefficientMatrix[num, num-2] = -wn_cal(n)*del_phi/wp * 1.j
    CoefficientMatrix[num, num] = -np.dot(vn_gen(n), K) * 1.j
    if n+2 <= J:
        CoefficientMatrix[num, num+2] = -wn_cal(n)*del_phi/wp * 1.j

In [20]:
from numpy import linalg as LA
eigenval, eigenmatrix = LA.eig(CoefficientMatrix)
tol = 1e-16

In [21]:
An0 = np.ones(2*J+1) # An(z=0)
for i in range(2*J+1):
    if (i-J)%2 == 0:
        An0[i] = 0
    else:
        An0[i] = 1/(abs((i-J)))**3

In [22]:
C = np.linalg.solve(eigenmatrix, An0) # coefficients
prop = np.multiply( np.exp(eigenval*L), C)

In [23]:
Bz = np.matmul(eigenmatrix, prop)

In [24]:
Bz1 = Bz[J+1]
Az1 = Bz1 * np.exp(1.j * ( (2*wn_cal(n)*del_phi/wp)- 0.5*(np.dot(vn_gen(1), K)) ))

In [30]:
np.absolute(Az1/An0[J+1])

1.0002790921075173